In [ ]:
import numpy as np
#tensorflow packages
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [ ]:
DataTraining,DataValidation,DataTest=tfds.load(name="imdb_reviews",split=('train[:60%]','train[60%:]','test'),as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWCYNWD/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWCYNWD/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWCYNWD/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
DataTraining

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
TrainingExampleBatch,TrainingLabelsBatch=next(iter(DataTraining.batch(10)))

In [ ]:
TrainingExampleBatch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
TrainingLabelsBatch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
embedding="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
HubLayer=hub.KerasLayer(embedding,input_shape=[],dtype=tf.string,trainable=True)

In [ ]:
HubLayer(TrainingExampleBatch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [ ]:
Model=tf.keras.Sequential()

In [ ]:
Model.add(HubLayer)

In [ ]:
Model.add(tf.keras.layers.Dense(16,activation='relu'))

In [ ]:
Model.add(tf.keras.layers.Dense(1))

In [ ]:
Model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
Model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
History=Model.fit(DataTraining.shuffle(10000).batch(100),epochs=25,validation_data=DataValidation.batch(100),verbose=1)

Epoch 1/25
150/150 [==============================] - 5s 24ms/step - loss: 0.5920 - accuracy: 0.6679 - val_loss: 0.5306 - val_accuracy: 0.7270
Epoch 2/25
150/150 [==============================] - 4s 23ms/step - loss: 0.4733 - accuracy: 0.7593 - val_loss: 0.4460 - val_accuracy: 0.7947
Epoch 3/25
150/150 [==============================] - 4s 27ms/step - loss: 0.3820 - accuracy: 0.8223 - val_loss: 0.3829 - val_accuracy: 0.8166
Epoch 4/25
150/150 [==============================] - 4s 23ms/step - loss: 0.3143 - accuracy: 0.8589 - val_loss: 0.3426 - val_accuracy: 0.8381
Epoch 5/25
150/150 [==============================] - 3s 20ms/step - loss: 0.2647 - accuracy: 0.8886 - val_loss: 0.3193 - val_accuracy: 0.8586
Epoch 6/25
150/150 [==============================] - 4s 23ms/step - loss: 0.2258 - accuracy: 0.9089 - val_loss: 0.3073 - val_accuracy: 0.8620
Epoch 7/25
150/150 [==============================] - 3s 20ms/step - loss: 0.2000 - accuracy: 0.9207 - val_loss: 0.3015 - val_accuracy: 0.8697

In [ ]:
Results=Model.evaluate(DataTest.batch(100),verbose=2)

250/250 - 4s - loss: 0.6853 - accuracy: 0.8390 - 4s/epoch - 15ms/step


In [ ]:
for Name,Value in zip(Model.metrics_names,Results):
  print("%s: %.3f"% (Name,Value))

loss: 0.685
accuracy: 0.839
